## Побудова бейзлайнового POS-таггера на основі pymorphy2

pymorphy2 не є POS-таггером: кожне слово аналізується у відриві від контексту і тільки якщо воно є у словнику. До того ж для української мови у pymorphy2 недоступний параметр score, який визначає імовірність правильності аналізу у випадку, коли для слова є кілька різних аналізів. Тим не менш, визначення частин мови у pymorphy2 можна покращити за допомогою кількох простих правил.

Правила формувались на тренувальній вибірці україномовних Universal Dependencies, тестувались на dev+test вибірках.

In [1]:
import conllu
import random
from collections import OrderedDict, Counter
import string
import gzip

from tokenize_uk import tokenize_words
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

from sklearn.metrics import classification_report, accuracy_score

In [2]:
fname = 'uk_iu-ud-train.conllu.gz'
with gzip.open(fname, 'rb') as f:
    raw_train = f.read().decode()

fname2 = 'uk_iu-ud-test.conllu.gz'
with gzip.open(fname2, 'rb') as f2:
    raw_test = f2.read().decode()
    
fname3 = 'uk_iu-ud-dev.conllu.gz'
with gzip.open(fname3, 'rb') as f3:
    raw_dev = f3.read().decode()

In [3]:
train = conllu.parse(raw_train)
test = conllu.parse(raw_test) + conllu.parse(raw_dev)

Найпростіший варіант - просто визначити відповіднити частин мови UD у тегах pymorphy2 (код звідси: https://github.com/vseloved/prj-nlp/blob/master/tasks/08-syntactic-parsing.md)

In [4]:
mapping = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PNCT": "PUNCT", "PRED": "ADV", "PREP": "ADP",
           "PRCL": "PART", None: "X"}

def normalize_pos(word):
    if word.tag.POS == "CONJ":
        if "coord" in word.tag:
            return "CCONJ"
        else:
            return "SCONJ"
    else:
        return mapping.get(word.tag.POS, word.tag.POS)

In [5]:
def classify_pymorphy(corpus):
    labels = []
    for sent in corpus:
        for i, w in enumerate(sent):
            word = w['form']
            wparsed = morph.parse(word.lower())[0]
            pos = normalize_pos(wparsed)
            labels.append(pos)
    return labels

In [6]:
true_labels = [w['upostag'] for sent in test for w in sent]
pred_labels = classify_pymorphy(test)

In [7]:
print(classification_report(true_labels, pred_labels))

             precision    recall  f1-score   support

        ADJ       0.92      0.90      0.91      2763
        ADP       0.99      0.59      0.74      2325
        ADV       0.69      0.61      0.65      1112
        AUX       0.00      0.00      0.00       183
      CCONJ       0.86      0.96      0.91       910
        DET       0.00      0.00      0.00       905
       INTJ       0.02      0.55      0.03        11
       NOUN       0.80      0.95      0.87      6757
        NUM       1.00      0.13      0.23       448
       PART       0.46      0.74      0.57       533
       PRON       0.36      0.68      0.47       749
      PROPN       0.00      0.00      0.00       939
      PUNCT       0.00      0.00      0.00      4664
      SCONJ       0.75      0.62      0.68       436
        SYM       0.00      0.00      0.00        19
       VERB       0.89      0.96      0.93      2366
          X       0.03      0.92      0.06       190

avg / total       0.60      0.61      0.59  

/home/igor/Bin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
print(round(accuracy_score(true_labels, pred_labels), 3))

0.612


Для покращення результату, потрібно:
- додати визначення знаків пунктуації (PUNCT) та особливих символів (SYM): pymorphy2 позначає їх як PNCT в тегу, але не в `word.tag.POS`;
- додати до числівників токени, які у `word.tag` (але не `word.tag.POS`) позначені як NUMB;
- додати визначення AUX та DET, які відсутні в pymorphy2 - просто через список слів (знайдених у тренувальній вибірці);
- додати просте правило для визначення PROPN (наприклад, якщо іменник не на початку речення починається з великої літери);
- додати списки поширених у тренувальній вибірці ADP, ADV тощо, з якими зараз pymorphy2 дуже плутається.

In [9]:
MAPPING = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PRED": "ADV", "PREP": "ADP", "PRCL": "PART"}

DET = ['інакший', 'його', 'тамтой', 'чий', 'їх', 'інш.', 'деякий', 'ввесь', 'ваш', 
     'ніякий', 'весь', 'інший', 'чийсь', 'жадний', 'другий', 'кожний', 
     'такий', 'оцей', 'скілька', 'цей', 'жодний', 'все', 'кілька', 'увесь', 
     'кожній', 'те', 'сей', 'ін.', 'отакий', 'котрий', 'усякий', 'самий', 
     'наш', 'усілякий', 'будь-який', 'сам', 'свій', 'всілякий', 'всенький', 'її', 
     'всякий', 'отой', 'небагато', 'який', 'їхній', 'той', 'якийсь', 'ин.', 'котрийсь', 
     'твій', 'мій', 'це', 'яка', 'якась', 'ця', 'якесь', 'яке', 'весь', 'самий']

SYM = set('#$%§©+=×÷=<>')
PUNCTUATION = set(string.punctuation) - SYM

def detect_pos(word, upper=False):
    """
    A function to detect POS of the word 
    using pymorphy2 and simple rules.
    """
    if not word.tag.POS:
        if word.word in PUNCTUATION:
            return "PUNCT"
        elif word.word in SYM:
            return 'SYM'
        elif 'NUMB' in word.tag:
            return 'NUM'
        else:
            return "X"
    elif word.tag.POS == "CONJ":
        if "coord" in word.tag:
            return "CCONJ"
        else:
            return "SCONJ"
    elif word.normal_form in DET:
        return 'DET'
    elif word.normal_form in ['на', 'за', 'після']:
        return 'ADP'
    elif word.normal_form in ['чи', 'натомість']:
        return 'CCONJ'
    elif word.normal_form in ['як', 'щоб', 'щоби', 'коли', 'мов',
                              'хоч', 'адже', 'тобто', 'ніби', 'хоча']:
        return 'SCONJ'
    elif word.normal_form in ['бути', 'б', 'би']:
        return 'AUX'
    elif word.normal_form in ['треба', 'завжди', 'потім', 'де', 'тоді', 
                              'там', 'тут', 'далі', 'тепер']:
        return 'ADV'
    elif 'NOUN' in word.tag and upper:
        return 'PROPN'
    else:
        return MAPPING.get(word.tag.POS, word.tag.POS)

def is_upper(word):
    if word[0].isupper():
        return True
    return False
    
def classify_pymorphy(corpus):
    labels = []
    for sent in corpus:
        for i, w in enumerate(sent):
            word = w['form']
            wparsed = morph.parse(word.lower())[0]
            if i == 0:
                pos = detect_pos(wparsed)
            else:
                pos = detect_pos(wparsed, upper=is_upper(word))
            labels.append(pos)
    return labels

In [10]:
true_labels = [w['upostag'] for sent in test for w in sent]
pred_labels = classify_pymorphy(test)

In [11]:
print(classification_report(true_labels, pred_labels))

             precision    recall  f1-score   support

        ADJ       0.92      0.89      0.91      2763
        ADP       0.99      0.82      0.90      2325
        ADV       0.87      0.69      0.77      1112
        AUX       0.77      0.82      0.79       183
      CCONJ       0.85      0.99      0.91       910
        DET       0.80      0.83      0.82       905
       INTJ       0.11      0.36      0.17        11
       NOUN       0.89      0.90      0.90      6757
        NUM       0.72      0.90      0.80       448
       PART       0.73      0.69      0.71       533
       PRON       0.79      0.56      0.66       749
      PROPN       0.71      0.85      0.77       939
      PUNCT       1.00      0.86      0.92      4664
      SCONJ       0.73      0.91      0.81       436
        SYM       0.17      0.42      0.24        19
       VERB       0.94      0.95      0.94      2366
          X       0.17      0.92      0.29       190

avg / total       0.90      0.86      0.87  

In [12]:
print(round(accuracy_score(true_labels, pred_labels), 3))

0.864


Остання спроба: використати ВЕСУМ для розрізнення різних типів займенників - "іменникових" (вони будуть PRON), "прислівникових" (вони, як правило, стають ADV) та "числівникових" і "прикметникових" (ці будуть DET). Для цього спершу витягнемо всі займенники зі словника разом із їхньою функціональною частиною мови, відповідним чином розфасуємо, а потім виділимо серед них тільки недвозначні (ті, для яких pymorphy2 має лише один варіант аналізу).

In [13]:
pron_dict = {}
with open('/mnt/hdd/Data/NLP/dict_corp_vis.txt') as f:
    for line in f.readlines():
        line = line.strip(' \n')
        word, tag = line.split()[:2]
        if '&pron' in tag:
            if (word in pron_dict.keys()
                and not pron_dict[word][1] == tag.split('&')[0].split(':')[0]):
                pron_dict[word] = 'AMBIGOUS'
                continue
            tagparts = tag.split('&')
            if tagparts[0].startswith('noun'):
                pron_dict[word] = ('PRON', 'noun')
            elif tagparts[0].startswith('adv'):
                pron_dict[word] = ('ADV', 'noun')
            elif tagparts[0].startswith('numr'):
                pron_dict[word] = ('DET', 'numr')
            elif tagparts[0].startswith('adj'):
                pron_dict[word] = ('DET', 'adj')
len(pron_dict.keys())

1754

In [14]:
disamb_dict = {}
for k,v in pron_dict.items():
    if v == 'AMBIGOUS':
        continue
    elif len(morph.parse(k)) > 1:
        if len(set(word.tag.POS for word in morph.parse(k))) > 1:
            continue
    else:
        disamb_dict[k] = pron_dict[k][0]
len(disamb_dict.keys())

697

In [15]:
def classify_pymorphy2(corpus):
    labels = []
    for sent in corpus:
        for i, w in enumerate(sent):
            word = w['form']
            wparsed = morph.parse(word.lower())[0]
            if word in disamb_dict.keys():
                pos = disamb_dict[word]
            elif i == 0:
                pos = detect_pos(wparsed)
            else:
                pos = detect_pos(wparsed, upper=is_upper(word))
            labels.append(str(pos))
    return labels

In [16]:
true_labels = [w['upostag'] for sent in test for w in sent]
pred_labels = classify_pymorphy2(test)
print(classification_report(true_labels, pred_labels))

             precision    recall  f1-score   support

        ADJ       0.92      0.89      0.91      2763
        ADP       0.99      0.82      0.90      2325
        ADV       0.87      0.71      0.78      1112
        AUX       0.77      0.82      0.79       183
      CCONJ       0.85      0.99      0.91       910
        DET       0.80      0.83      0.82       905
       INTJ       0.11      0.36      0.17        11
       NOUN       0.89      0.90      0.90      6757
        NUM       0.72      0.90      0.80       448
       PART       0.73      0.69      0.71       533
       PRON       0.82      0.56      0.67       749
      PROPN       0.71      0.85      0.77       939
      PUNCT       1.00      0.86      0.92      4664
      SCONJ       0.73      0.91      0.81       436
        SYM       0.17      0.42      0.24        19
       VERB       0.94      0.95      0.94      2366
          X       0.17      0.92      0.29       190

avg / total       0.90      0.86      0.87  

In [17]:
print(round(accuracy_score(true_labels, pred_labels), 3))

0.864


In [18]:
def combine_tags(tag_list):
    mapping = {'DET': 'PRON', 'SYM': 'PUNCT', 
               'AUX': 'VERB', 'PROPN': 'NOUN'}
    return [mapping.get(tag, tag) for tag in tag_list]

print(round(accuracy_score(combine_tags(true_labels), combine_tags(pred_labels)), 3))

0.888


86%-89% - це значне покращення порівняно з першим варіантом, але для POS-таггера досить поганий результат. На корпусі WSJ навіть бейзлайнові класифікатори дають точність 92%, а state-of-the-art - 97%.